### Import necessary Library

In [ ]:
pip install openpyxl

In [ ]:
pip install opencv-python

In [ ]:
pip install alphashape

In [ ]:
pip install numpy==1.26.4 -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
%run 'functions_lib/pt_generator.py'
%run 'model.py'
import numpy as np
from functions_lib.pt_generator import *
from model import *

We note that now we set parameter $\mu=1$. If you want to get three characterization metrics when $\mu\neq1$, file "pt_generator" provides adjustable parameter.

### Generating 5 types of data:
    1. pt_uncorrelated_train -- work as training set in pretraining phase
    2. pt_uncorrelated_validation -- work as validation set in pretraining phase
    3. pt_correlated_train -- work as training set in training phase
    4. pt_correlated_validation -- work as validation set in training phase
    5. pt_correlated_output -- work as output set in output phase

In [ ]:
correlated(n=100000,excel_path='/home/featurize/work/QPCNN/Dataset/pt_correlated_train.xlsx',mode=2)
correlated(n=4000,excel_path='/home/featurize/work/QPCNN/Dataset/pt_correlated_validation.xlsx',mode=2)
correlated(n=10000,excel_path='/home/featurize/work/QPCNN/Dataset/pt_correlated_output.xlsx',mode=5)
uncorrelated(n=10000,excel_path='/home/featurize/work/QPCNN/Dataset/pt_uncorrelated_train.xlsx')
uncorrelated(n=4000,excel_path='/home/featurize/work/QPCNN/Dataset/pt_uncorrelated_validation.xlsx')

### Simulated experiment

In [ ]:
count(N = 100000,input_file = '/home/featurize/work/QPCNN/Dataset/pt_correlated_train.xlsx',
      output_file = '/home/featurize/work/QPCNN/Dataset/pm_correlated_train.xlsx',if_count = False)

count(N = 100000,input_file = '/home/featurize/work/QPCNN/Dataset/pt_correlated_validation.xlsx',
      output_file = '/home/featurize/work/QPCNN/Dataset/pm_correlated_validation.xlsx',if_count = False)

count(N = 100000,input_file = '/home/featurize/work/QPCNN/Dataset/pt_correlated_output.xlsx',
      output_file = '/home/featurize/work/QPCNN/Dataset/pm_correlated_output.xlsx',if_count = True)

count(N = 100000,input_file = '/home/featurize/work/QPCNN/Dataset/pt_uncorrelated_train.xlsx',
      output_file = '/home/featurize/work/QPCNN/Dataset/pm_uncorrelated_train.xlsx',if_count = False)

count(N = 100000,input_file = '/home/featurize/work/QPCNN/Dataset/pt_uncorrelated_validation.xlsx',
      output_file = '/home/featurize/work/QPCNN/Dataset/pm_uncorrelated_validation.xlsx',if_count = False)

shuffle_excel('/home/featurize/work/QPCNN/Dataset/pm_correlated_train.xlsx')

shuffle_excel('/home/featurize/work/QPCNN/Dataset/pm_uncorrelated_train.xlsx')

### Pretraining phase

In [ ]:
cur_date, cur_hour, cur_min = get_time()
hp_pretrain = HyperParameters_pretrain('/home/featurize/work/QPCNN',cur_date, cur_hour, cur_min)

for folder1 in ['QPCNN_log', 'QPCNN_saved', 'QPCNN_output']:#log,saved,output.
    os.makedirs('{}/output_Data/{}/'.format(hp_pretrain.save_path,folder1), exist_ok=True)#exist_ok = True means the existence of this file is OK 
    if folder1 == 'QPCNN_output':#Inference means output, Validation is not used in our work. 
        for folder2 in ['validation', 'inference']:
            os.makedirs('{}/output_Data/{}/{}/'.format(hp_pretrain.save_path,folder1,folder2), exist_ok=True)

# Log Setting 
logging.basicConfig(
    filename=hp_pretrain.log_path,#Create a FileHandler with a file name called hp.log_path, that is, log information to the file hp.log_path
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s',#Time - Level - Information
    datefmt='%m-%d %H:%M',#Month - Date Hour: minute
    level=logging.INFO#info
)

# Check whether the system can use a GPU for calculation. If no GPU is available, use a CPU
hp_pretrain.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#在日志中记录训练的参数
logging.info('Iteration times in this training: {}'.format(hp_pretrain.iteration))  # Record the number of iterations for this training
logging.info('Computational Device: {}'.format(hp_pretrain.device))  # The processor (CPU or GPU) used in the calculation is recorded in the log
logging.info('Learning parameters: lr-{}, lr_decay-{}, lr_patience-{}, early_patience-{}'.format(hp_pretrain.lr, hp_pretrain.lr_decay, hp_pretrain.lr_patience, hp_pretrain.early_patience))
logging.info('#' * 50)

# Model
qpcnn = QPCNN()  # 
qpcnn.to(hp_pretrain.device) #CPU or GPU
optimizer = optim.Adam(qpcnn.parameters(), lr=hp_pretrain.lr)  # Adam optimizer,
# optimizer = optim.SGD(spircnet.parameters(), lr=1e-3, momentum=0.99)  # SGD optimizer

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=hp_pretrain.lr_decay, patience=hp_pretrain.lr_patience,
                                                    verbose=True, threshold=hp_pretrain.threshold, min_lr=hp_pretrain.min_lr)  # learning rate scheduler
# early_stopping = EarlyStopping(patience=hp.early_patience, delta=hp.min_delta, verbose=False, path=hp.model_save_path)

since = time.time()  

set_file(hp_pretrain.save_path) #Convert excel data in a file package to numpy for easy operation

data_input_train = np.load(hp_pretrain.data_train)  # Loading training data
input_vector_train, qubit_vector_train, qubit_spin_train, input_spin_train = data_prepartion_trainandtest(data_input_train)
instance_train= qubit_vector_train.shape[0]

data_input_test = np.load(hp_pretrain.data_test)  # Loading testing data
input_vector_test, qubit_vector_test, qubit_spin_test,input_spin_test = data_prepartion_trainandtest(data_input_test)
instance_test = input_vector_test.shape[0]

qpcnn = torch.nn.DataParallel(qpcnn)    #Multi-GPU acceleration
criterion_MSE_train = nn.MSELoss()  # Loss Function MAE(L1)
criterion_MAE_train = nn.L1Loss()  # Loss Function MSE
MSE_plot = []   # Draw an empty array and load MSE, MAE and Shannon entropy data
MAE_plot = []   
entropy_plot = [] 

In [ ]:
for iter in range(hp_pretrain.iteration):

    for each_instance_train in range(0,instance_train):
        #============================================================================================
        if each_instance_train % hp_pretrain.test_log == 0:
            loss_test_summary_MAE = 0
            loss_test_summary_MSE = 0
            summary_entropy = 0
            criterion_MAE_test = nn.L1Loss()
            criterion_MSE_test = nn.MSELoss()
            for each_instance_test in range(instance_test):
                output_spin_test = qpcnn(input_vector_test[each_instance_test,:,:].to(hp_pretrain.device), qubit_vector_test[each_instance_test,:,:].to(hp_pretrain.device), qubit_spin_test[each_instance_test,:,:].to(hp_pretrain.device))
                loss_test_MSE = criterion_MSE_test(output_spin_test[0,:,:].to(hp_pretrain.device), input_spin_test[each_instance_test,:,:].to(hp_pretrain.device)).data  
                loss_test_summary_MSE += loss_test_MSE
                loss_test_MAE = criterion_MAE_test(output_spin_test[0,:,:].to(hp_pretrain.device), input_spin_test[each_instance_test,:,:].to(hp_pretrain.device)).data  
                loss_test_summary_MAE += loss_test_MAE
            loss_test_summary_MSE = loss_test_summary_MSE/instance_test
            loss_test_summary_MAE = loss_test_summary_MAE/instance_test
            MSE_plot.append(loss_test_summary_MSE.cpu().numpy())#MSE
            MAE_plot.append(loss_test_summary_MAE.cpu().numpy())#MAE
            lr_scheduler.step(loss_test_summary_MSE)
            #============================================================================================
            logging.info('loss_test_MSE ={}'.format(loss_test_summary_MSE))
            print('iter:{},epoch:{},validation number:{},MSE:{},MAE:{},entropy:{}'.format(iter,each_instance_train,instance_test,loss_test_summary_MSE,loss_test_summary_MAE,summary_entropy))   

        qpcnn.train() 
        optimizer.zero_grad() 
        output_spin_train = qpcnn(input_vector_train[each_instance_train,:,:].to(hp_pretrain.device), qubit_vector_train[each_instance_train,:,:].to(hp_pretrain.device), qubit_spin_train[each_instance_train,:,:].to(hp_pretrain.device))
        loss_train = criterion_MSE_train(output_spin_train[0,:,:].to(hp_pretrain.device), input_spin_train[each_instance_train,:,:].to(hp_pretrain.device)) 
        loss_train.backward()  
        optimizer.step()  

        if each_instance_train == instance_train-1:
            loss_test_summary_MAE = 0
            loss_test_summary_MSE = 0
            summary_entropy = 0
            criterion_MAE_test = nn.L1Loss()
            criterion_MSE_test = nn.MSELoss()
            for each_instance_test in range(instance_test):
                output_spin_test = qpcnn(input_vector_test[each_instance_test,:,:].to(hp_pretrain.device), qubit_vector_test[each_instance_test,:,:].to(hp_pretrain.device), qubit_spin_test[each_instance_test,:,:].to(hp_pretrain.device))
                loss_test_MSE = criterion_MSE_test(output_spin_test[0,:,:].to(hp_pretrain.device), input_spin_test[each_instance_test,:,:].to(hp_pretrain.device)).data
                loss_test_summary_MSE += loss_test_MSE
                loss_test_MAE = criterion_MAE_test(output_spin_test[0,:,:].to(hp_pretrain.device), input_spin_test[each_instance_test,:,:].to(hp_pretrain.device)).data  
                loss_test_summary_MAE += loss_test_MAE
            loss_test_summary_MSE = loss_test_summary_MSE/instance_test
            loss_test_summary_MAE = loss_test_summary_MAE/instance_test
            MSE_plot.append(loss_test_summary_MSE.cpu().numpy())#MSE
            MAE_plot.append(loss_test_summary_MAE.cpu().numpy())#MAE           
            lr_scheduler.step(loss_test_summary_MSE)  
            print('iter:{},epoch:{},validation number:{},MSE:{},MAE:{},entropy:{}'.format(iter,each_instance_train,instance_test,loss_test_summary_MSE,loss_test_summary_MAE,summary_entropy))

    if iter % hp_pretrain.train_log == 0:
        print("---iter{} finished---".format(iter))
        logging.info('iter {}, train loss: {}'.format(iter, loss_train.data))
        logging.info('*' * 50)
if hp_pretrain.logging_parameter == True:
    checkpoint = {'net': qpcnn.state_dict(), 'optimizer': optimizer.state_dict(),
              'lr_scheduler': lr_scheduler.state_dict()}
    torch.save(checkpoint, hp_pretrain.model_save_path)  

### Training phase and output phase

In [ ]:
cur_date, cur_hour, cur_min = get_time()
hp_output = HyperParameters_output('/home/featurize/work/QPCNN',cur_date, cur_hour, cur_min,'/home/featurize/work/QPCNN/output_Data/QPCNN_saved/2025-08-11-19h35min.pth')

# Folder creation
for folder1 in ['QPCNN_log', 'QPCNN_saved', 'QPCNN_output']:#log,saved,output.
    os.makedirs('{}/output_Data/{}/'.format(hp_output.save_path,folder1), exist_ok=True)#exist_ok = True means the existence of this file is OK 
    if folder1 == 'QPCNN_output':#Inference means output, Validation is not used in our work. 
        for folder2 in ['validation', 'inference']:
            os.makedirs('{}/output_Data/{}/{}/'.format(hp_output.save_path,folder1,folder2), exist_ok=True)

# Log Setting 
logging.basicConfig(
    filename=hp_output.log_path,#Create a FileHandler with a file name called hp.log_path, that is, log information to the file hp.log_path
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s',#Time - Level - Information
    datefmt='%m-%d %H:%M',#Month - Date Hour: minute
    level=logging.INFO#info
)

#================================================================================= importing pt and pm for calculating pn
pm = pd.read_excel("{}/Dataset/pm_correlated_output.xlsx".format(hp_output.save_path))
pm = pm.to_numpy()
pm_0 = pm[:,7]
pm_1 = pm[:,8]

pt = pd.read_excel("{}/Dataset/pt_correlated_output.xlsx".format(hp_output.save_path))
pt = pt.to_numpy()
pt_trans = np.zeros((2*pt.shape[0],2))
for i in range(pt.shape[0]):
    pt_trans[i,0] = pt[i,6]/(pt[i,6]+pt[i,7])
    pt_trans[i,1] = pt[i,7]/(pt[i,6]+pt[i,7])
    pt_trans[i+pt.shape[0],0] = pt[i,8]/(pt[i,8]+pt[i,9])
    pt_trans[i+pt.shape[0],1] = pt[i,9]/(pt[i,8]+pt[i,9])
pt_0 = pt_trans[:,0]
pt_1 = pt_trans[:,1]
#============================================================================================================

# Check whether the system can use a GPU for calculation. If no GPU is available, use a CPU
hp_output.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

logging.info('Iteration times in this training: {}'.format(hp_output.iteration))  # Record the number of iterations for this training
logging.info('Computational Device: {}'.format(hp_output.device))  # The processor (CPU or GPU) used in the calculation is recorded in the log
logging.info('Learning parameters: lr-{}, lr_decay-{}, lr_patience-{}, early_patience-{}'.format(hp_output.lr, hp_output.lr_decay, hp_output.lr_patience, hp_output.early_patience))
logging.info('#' * 50)

# Model
qpcnn = QPCNN()  # 
qpcnn.to(hp_output.device) #CPU or GPU
optimizer = optim.Adam(qpcnn.parameters(), lr=hp_output.lr)  # Adam optimizer,
# optimizer = optim.SGD(spircnet.parameters(), lr=1e-3, momentum=0.99)  # SGD optimizer

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=hp_output.lr_decay, patience=hp_output.lr_patience,
                                                    verbose=True, threshold=hp_output.threshold, min_lr=hp_output.min_lr)  # learning rate scheduler
early_stopping = EarlyStopping(patience=hp_output.early_patience, delta=hp_output.min_delta, verbose=False, path=hp_output.model_save_path)

if hp_output.training_resume == True:  # inheriting the model
    checkpoint = torch.load(hp_output.model_load_path)  # loading
    qpcnn = torch.nn.DataParallel(qpcnn).cuda() #Multi-card parallel nn.DataParallel data previously reserved for multi-card parallel data
    qpcnn.load_state_dict(checkpoint['net'])  
    optimizer.load_state_dict(checkpoint['optimizer'])  
    #lr_scheduler.load_state_dict(checkpoint['lr_scheduler']) 
    logging.info('Load and continue training the existing model: {}'.format(hp_output.model_load_path))
    for param_group in optimizer.param_groups:
        param_group['lr'] = hp_output.lr #Go back to the original learning rate
else:  
    logging.info('New created model')
#============================================================================================================

since = time.time()  
set_file(hp_output.save_path)#Convert excel data in a file package to numpy for easy operation
data_input_train = np.load(hp_output.data_train)  # Loading training data
input_vector_train, qubit_vector_train, qubit_spin_train, input_spin_train = data_prepartion_trainandtest(data_input_train) # 处理训练集数据
instance_train= qubit_vector_train.shape[0]

data_input_test = np.load(hp_output.data_test)  # Loading testing data
input_vector_test, qubit_vector_test, qubit_spin_test,input_spin_test = data_prepartion_trainandtest(data_input_test) # 处理训练集数据
instance_test = input_vector_test.shape[0]

qpcnn = torch.nn.DataParallel(qpcnn)    #Multi-GPU acceleration
criterion_MSE_train = nn.MSELoss()  # Loss Function MAE(L1)
criterion_MAE_train = nn.L1Loss()  # Loss Function MSE
MSE_plot = []   # Draw an empty array and load MSE, MAE and Shannon entropy data
MAE_plot = []   
entropy_plot = [] 
epoch=0

In [ ]:
for iter in range(hp_output.iteration):

    for each_instance_train in range(0,instance_train):# batch = 1
        #=============================================Calculate MSE, MAE before each training===============================================
        if each_instance_train % hp_output.test_log == 0:
            loss_test_summary_MAE = 0
            loss_test_summary_MSE = 0
            summary_entropy = 0
            criterion_MAE_test = nn.L1Loss()
            criterion_MSE_test = nn.MSELoss()
            for each_instance_test in range(instance_test):
                output_spin_test = qpcnn(input_vector_test[each_instance_test,:,:].to(hp_output.device), qubit_vector_test[each_instance_test,:,:].to(hp_output.device), qubit_spin_test[each_instance_test,:,:].to(hp_output.device))
                loss_test_MSE = criterion_MSE_test(output_spin_test[0,:,:].to(hp_output.device), input_spin_test[each_instance_test,:,:].to(hp_output.device)).data  
                loss_test_summary_MSE += loss_test_MSE
                loss_test_MAE = criterion_MAE_test(output_spin_test[0,:,:].to(hp_output.device), input_spin_test[each_instance_test,:,:].to(hp_output.device)).data  
                loss_test_summary_MAE += loss_test_MAE
            loss_test_summary_MSE = loss_test_summary_MSE/instance_test
            loss_test_summary_MAE = loss_test_summary_MAE/instance_test
            MSE_plot.append(loss_test_summary_MSE.cpu().numpy())#MSE
            MAE_plot.append(loss_test_summary_MAE.cpu().numpy())#MAE
            lr_scheduler.step(loss_test_summary_MSE)  # Based on loss_test_summary_MAE to adjust Learning rate
            logging.info('loss_test_MSE ={}'.format(loss_test_summary_MSE))
            print('loss_test_MSE ={},loss_test_MAE ={}'.format(loss_test_summary_MSE,loss_test_summary_MAE))

            
            #=============================================output phase:===============================================
            data_input_infer = np.load(hp_output.data_infer)  
            data_input_infer = data_input_infer[:,:7]
            input_vector_infer, qubit_vector_infer, qubit_spin_infer = data_prepartion_infer(data_input_infer) 
            instance_infer = input_vector_infer.shape[0]
            output_summary_infer = np.zeros((instance_infer,2))
            for each_instance_infer in range(instance_infer):
                output_spin_infer = qpcnn(input_vector_infer[each_instance_infer,:,:].to(hp_output.device), qubit_vector_infer[each_instance_infer,:,:].to(hp_output.device), qubit_spin_infer[each_instance_infer,:,:].to(hp_output.device)) #shape(1,1,8)
                output_spin_infer = output_spin_infer.view(1,2)
                output_spin_infer = output_spin_infer.detach().cpu().numpy() 
                
                po_0 = output_spin_infer[0,1]*pt_0[each_instance_infer]*pm_0[each_instance_infer] / (output_spin_infer[0,0]*pt_1[each_instance_infer]*pm_1[each_instance_infer] + output_spin_infer[0,1]*pt_0[each_instance_infer]*pm_0[each_instance_infer])
                po_0_clipped = np.clip(po_0, 1e-10, 1 - 1e-10)# ensure p != 0
                summary_entropy = summary_entropy - po_0_clipped * np.log(po_0_clipped) - (1 - po_0_clipped) * np.log(1 - po_0_clipped)
                # output_spin_infer[0,0] = po_0_clipped
                # output_spin_infer[0,1] = 1 - po_0_clipped
                output_summary_infer[each_instance_infer,:] = output_spin_infer # now we output pn, if you want to output po(n->infinite), use the last two lines of code

            summary_entropy = summary_entropy / instance_infer
            entropy_plot.append(summary_entropy)#entropy
            #print(output_spin_infer.shape)  #torch.Size([1, 1, 2])
            print('iter:{},epoch:{},validation number:{},MSE:{},MAE:{},entropy:{}'.format(iter,each_instance_train,instance_test,loss_test_summary_MSE,loss_test_summary_MAE,summary_entropy))
            logging.info('iter:{},epoch:{},validation number:{},MSE:{},MAE:{},entropy:{}'.format(iter,each_instance_train,instance_test,loss_test_summary_MSE,loss_test_summary_MAE,summary_entropy))

            output_summary_infer = np.concatenate((data_input_infer[:,:7],output_summary_infer),axis=1)
            np.save(hp_output.output_save_path + '/inference/output.npy', output_summary_infer)  # save
            colname = ['input_a1_x','input_a1_y','input_a1_z',
                                                    'qubit_a1_x','qubit_a1_y','qubit_a1_z',
                                                    'qubit_a1_spin',
                                                    '0','1']
            
            pd.DataFrame(output_summary_infer,columns = colname).to_excel(hp_output.output_save_path + '/inference/output_iter{}_epoch{}.xlsx'.format(iter,epoch),index=True,
                                        columns = colname)
            epoch = epoch+1
            print('iter:{},epoch:{},output number:{}'.format(iter,each_instance_train,instance_infer))    

        qpcnn.train() 
        optimizer.zero_grad() # grading resetting
        output_spin_train = qpcnn(input_vector_train[each_instance_train,:,:].to(hp_output.device), qubit_vector_train[each_instance_train,:,:].to(hp_output.device), qubit_spin_train[each_instance_train,:,:].to(hp_output.device))
        loss_train = criterion_MSE_train(output_spin_train[0,:,:].to(hp_output.device), input_spin_train[each_instance_train,:,:].to(hp_output.device)) 
        loss_train.backward()  # (Calculate the gradient value of the parameter) by backpropagation
        optimizer.step()  # Update parameters (by gradient descent)

        if each_instance_train == instance_train-1:#last
            #sfwml.eval()    
            loss_test_summary_MAE = 0
            loss_test_summary_MSE = 0
            summary_entropy = 0
            criterion_MAE_test = nn.L1Loss()
            criterion_MSE_test = nn.MSELoss()
            for each_instance_test in range(instance_test):
                output_spin_test = qpcnn(input_vector_test[each_instance_test,:,:].to(hp_output.device), qubit_vector_test[each_instance_test,:,:].to(hp_output.device), qubit_spin_test[each_instance_test,:,:].to(hp_output.device))
                loss_test_MSE = criterion_MSE_test(output_spin_test[0,:,:].to(hp_output.device), input_spin_test[each_instance_test,:,:].to(hp_output.device)).data  
                loss_test_summary_MSE += loss_test_MSE
                loss_test_MAE = criterion_MAE_test(output_spin_test[0,:,:].to(hp_output.device), input_spin_test[each_instance_test,:,:].to(hp_output.device)).data 
                loss_test_summary_MAE += loss_test_MAE
            loss_test_summary_MSE = loss_test_summary_MSE/instance_test
            loss_test_summary_MAE = loss_test_summary_MAE/instance_test
            MSE_plot.append(loss_test_summary_MSE.cpu().numpy())#MSE
            MAE_plot.append(loss_test_summary_MAE.cpu().numpy())#MAE
            lr_scheduler.step(loss_test_summary_MSE)  
            
            #=============================================The same as previous work===============================================
            data_input_infer = np.load(hp_output.data_infer)  # 
            input_vector_infer, qubit_vector_infer, qubit_spin_infer = data_prepartion_infer(data_input_infer)
            instance_infer = input_vector_infer.shape[0]
            output_summary_infer = np.zeros((instance_infer,2))
            for each_instance_infer in range(instance_infer):
                output_spin_infer = qpcnn(input_vector_infer[each_instance_infer,:,:].to(hp_output.device), qubit_vector_infer[each_instance_infer,:,:].to(hp_output.device), qubit_spin_infer[each_instance_infer,:,:].to(hp_output.device)) #shape(1,1,8)
                output_spin_infer = output_spin_infer.view(1,2)
                output_spin_infer = output_spin_infer.detach().cpu().numpy() 
                
                po_0 = output_spin_infer[0,1]*pt_0[each_instance_infer]*pm_0[each_instance_infer] / (output_spin_infer[0,0]*pt_1[each_instance_infer]*pm_1[each_instance_infer] + output_spin_infer[0,1]*pt_0[each_instance_infer]*pm_0[each_instance_infer])
                po_0_clipped = np.clip(po_0, 1e-10, 1 - 1e-10)
                summary_entropy = summary_entropy - po_0_clipped * np.log(po_0_clipped) - (1 - po_0_clipped) * np.log(1 - po_0_clipped)
                # output_spin_infer[0,0] = po_0_clipped
                # output_spin_infer[0,1] = 1 - po_0_clipped
                output_summary_infer[each_instance_infer,:] = output_spin_infer

            summary_entropy = summary_entropy / instance_infer
            entropy_plot.append(summary_entropy)#entropy
            #print(output_spin_infer.shape)  #torch.Size([1, 1, 2])
            print('iter:{},epoch:{},validation number:{},MSE:{},MAE:{},entropy:{}'.format(iter,each_instance_train,instance_test,loss_test_summary_MSE,loss_test_summary_MAE,summary_entropy))
            output_summary_infer = np.concatenate((data_input_infer[:,:7],output_summary_infer),axis=1)
            np.save(hp_output.output_save_path + '/inference/output.npy', output_summary_infer)  
            colname = ['input_a1_x','input_a1_y','input_a1_z',
                                                    'qubit_a1_x','qubit_a1_y','qubit_a1_z',
                                                    'qubit_a1_spin',
                                                    '0','1']
            
            pd.DataFrame(output_summary_infer,columns = colname).to_excel(hp_output.output_save_path + '/inference/output_iter{}_epoch{}.xlsx'.format(iter,epoch),index=True,
                                        columns = colname)
            print('iter:{},epoch:{},output number:{}'.format(iter,each_instance_train,instance_infer))

    if iter % hp_output.train_log == 0:# loss_train is recorded every hp.train_log iter
        print("---iter{} finished---".format(iter))
        logging.info('iter {}, train loss: {}'.format(iter, loss_train.data))
        logging.info('*' * 50)
#=================================================saving the model====================================================
if hp_output.logging_parameter == True:
    checkpoint = {'net': qpcnn.state_dict(), 'optimizer': optimizer.state_dict(),
              'lr_scheduler': lr_scheduler.state_dict()}
    torch.save(checkpoint, hp_output.model_save_path)  

### Generatoring $f$, $p^o$ and Shannon entropy

In [ ]:
pip install numpy==2.3.2 -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
%run 'functions_lib/exposeddata_generator.py'
from functions_lib.exposeddata_generator import *

In [ ]:

calculating_f('/home/featurize/work/QPCNN')

In [ ]:
calculating_p_o('/home/featurize/work/QPCNN')

In [ ]:
calculating_entropy('/home/featurize/work/QPCNN') 
for i in [0,1,2,5,20,100,101]: # We note 101 as unfrabricated data
    polytopes('/home/featurize/work/QPCNN',i)
    roundness_cal('/home/featurize/work/QPCNN', i)

### PDM

In [ ]:
for i in [0,1,5,20,100,101]:
    real_experiment(file_path = '/home/featurize/work/QPCNN', times = i)